In [2]:
import pygame as pg
import numpy as np
import random as rd 
pg.init()

class HealingAgent: #basically contains all the healing agents. its state and its healing abilities.
    def __init__(self, screen, cell_size, x, y, b):
        self.screen = screen
        self.cell_size = cell_size
        self.row = int(y / cell_size)
        self.col = int(x / cell_size)
        self.state = 'None'# None = wandering, Focused = moving to target
        self.target = None #there's no target assigned. Null value assigned
        self.grid = b
        self.rows = len(b)

    def sense_damage(self, radius=1): #the agent senses damage in nearby area
        for i in range(-radius, radius + 1):
            for j in range(-radius, radius + 1):
                nr, nc = self.row + i, self.col + j #row and column number
                if 0 <= nr < self.rows and 0 <= nc < self.rows:
                    if self.grid[nr][nc] > 0.5:  # unhealthy if red > 127
                        self.target = (nr, nc)
                        self.state = 'Focused'#now we know which cell is damaged, no need to move randomly
                        return

    def move(self):
        if self.state == 'None': #wandering
            self.row += rd.choice([-1, 0, 1]) #makes any random choice, right left or same row
            self.col += rd.choice([-1, 0, 1]) #makes any random choice, right left or same col
        elif self.state == 'Focused' and self.target:
            tr, tc = self.target
            dr = np.sign(tr - self.row)
            dc = np.sign(tc - self.col)

            # small chance of randomness still
            self.row += rd.choices([dr, rd.choice([-1, 0, 1])], weights=[0.8, 0.2])[0]
            self.col += rd.choices([dc, rd.choice([-1, 0, 1])], weights=[0.8, 0.2])[0]

        # Keep within grid bounds
        self.row = max(0, min(self.rows - 1, self.row))
        self.col = max(0, min(self.rows - 1, self.col))

    def act(self):
        self.sense_damage()
        self.move()

        if self.grid[self.row][self.col] > 0.5:
            pg.draw.rect(self.screen, (0, 0, 255), pg.Rect(self.col * self.cell_size, self.row * self.cell_size, self.cell_size, self.cell_size))
            pg.display.update()
            pg.time.delay(500)
            self.grid[self.row][self.col] = 0  # healed
            pg.draw.rect(self.screen, (0, 255, 0), pg.Rect(self.col * self.cell_size, self.row * self.cell_size, self.cell_size, self.cell_size))
            pg.draw.rect(self.screen, (0, 0, 0), pg.Rect(self.col * self.cell_size, self.row * self.cell_size, self.cell_size, self.cell_size), 1)
            pg.display.update()
            self.state = 'None'
            self.target = None

def drawgrid(screen, rows, cell_size, b):
    for row in range(rows):
        for column in range(rows):
            health = b[row][column]
            red = int(255 * health)
            green = int(255 * (1 - health))
            color = (red, green, 0)
            pg.draw.rect(screen, color, pg.Rect(column * cell_size, row * cell_size, cell_size, cell_size))
            pg.draw.rect(screen, (0, 0, 0), pg.Rect(column * cell_size, row * cell_size, cell_size, cell_size), 1)
    pg.display.update()

def updatedgrid(screen, cell_size, change, x, y, b):
    col_index = int(x / cell_size)
    row_index = int(y / cell_size)
    color = screen.get_at((x, y))[:3]
    health = color[0] / 255.0

    if health > 0.5:
        new_health = max(0, health - change / 1000)
    else:
        new_health = min(1, health + change / 1000)

    b[row_index][col_index] = new_health
    red = int(255 * new_health)
    green = int(255 * (1 - new_health))
    color = (red, green, 0)

    pg.draw.rect(screen, color, pg.Rect(col_index * cell_size, row_index * cell_size, cell_size, cell_size))
    pg.draw.rect(screen, (0, 0, 0), pg.Rect(col_index * cell_size, row_index * cell_size, cell_size, cell_size), 1)
    pg.display.update()

def main():
    rows = 10
    window_size = 500
    cell_size = window_size / rows
    screen = pg.display.set_mode((window_size, window_size))
    pg.display.set_caption("Tissue Sample")

    a = np.random.randint(0, 2, size=100)
    b = a.reshape((rows, rows)).astype(float)
    drawgrid(screen, rows, cell_size, b)

    agents = []
    change = 100
    running = True

    while running:
        for event in pg.event.get():
            if event.type == pg.QUIT:
                running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                (x, y) = pg.mouse.get_pos()
                updatedgrid(screen, cell_size, change, x, y, b)
                healer = HealingAgent(screen, cell_size, x, y, b)
                agents.append(healer)

        for healer in agents:
            healer.act()

        pg.time.delay(100)

    pg.quit()

if __name__ == "__main__":
    main()
